## Building Machine Learning Classifiers: Building a basic Random Forest model

In [26]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier()


### Explore RandomForestClassifier through Cross-Validation

In [6]:
from sklearn.model_selection import KFold, cross_val_score

In [7]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97755835, 0.98025135, 0.97574124, 0.96675651, 0.97124888])

### Explore RandomForestClassifier through Holdout Set

In [8]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [9]:
data['label']

0       spam
1        ham
2        ham
3        ham
4        ham
        ... 
5562    spam
5563     ham
5564     ham
5565     ham
5566     ham
Name: label, Length: 5567, dtype: object

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [15]:
sorted(zip(rf_model.feature_importances_,X_train.columns), reverse = True)[0:10]

[(0.07182609036568678, 'body_len'),
 (0.038758107183470815, 4796),
 (0.03133129814777587, 1803),
 (0.026354356658755625, 6285),
 (0.02635028333159867, 7350),
 (0.02621221495835718, 2031),
 (0.023147055755986384, 5724),
 (0.02104466125040562, 5988),
 (0.020975942268386807, 3134),
 (0.01744709424712511, 7027)]

In [17]:
y_pred = rf_model.predict(X_test)
precision,recall,fscore,support = score(y_test, y_pred, pos_label='spam', average='binary')

In [18]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.603 / Accuracy: 0.944


### Build our own Grid-search

#### Grid-search: Exhaustively search all parameter combinations in a given grid to determine the best model.

#### Cross-validation: Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

In [19]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [20]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.224 / Accuracy: 0.891
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.59 / Accuracy: 0.943
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.667 / Accuracy: 0.953
Est: 10 / Depth: None ---- Precision: 0.984 / Recall: 0.769 / Accuracy: 0.966
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.288 / Accuracy: 0.9
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.622 / Accuracy: 0.947
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.705 / Accuracy: 0.959
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.814 / Accuracy: 0.974
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.224 / Accuracy: 0.891
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.596 / Accuracy: 0.943
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.731 / Accuracy: 0.962
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.795 / Accuracy: 0.971


### Exploring parameter settings using GridSearchCV

In [21]:
from sklearn.model_selection import GridSearchCV

In [27]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,55.937852,5.974114,0.523059,0.114714,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.977558,0.974843,0.968553,0.972147,0.973953,0.003272,1
7,29.634321,0.234484,0.420710,0.015058,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.977558,0.973944,0.966757,0.971249,0.973593,0.004283,2
8,56.700225,0.701750,0.629107,0.035857,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.977558,0.973944,0.966757,0.970350,0.973413,0.004395,3
10,31.417748,0.559549,0.401475,0.012988,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.974865,0.973944,0.967655,0.969452,0.972336,0.003192,4
6,3.454111,0.120623,0.228487,0.015515,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.976661,0.973968,0.974843,0.964960,0.969452,0.971977,0.004236,5


In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

### Explore GradientBoostingClassifier Attributes & Hyperparameters

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_initialized', '_check_n_features', '_check_params', '_clear_state', '_compute_partial_dependence_recursion', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_more_tags', '_raw_predict', '_raw_predict_init', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_data', '_validate_estimator', '_validate_y', '_warn_mae_for_criterion', 'apply

### Build our own Grid-search

In [31]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [32]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\abhil\anaconda3\envs\KerasPython37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.86
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.965 / Recall: 0.699 / Accuracy: 0.954
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.932 / Recall: 0.795 / Accuracy: 0.963
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.006 / Accuracy: 0.861
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.927 / Recall: 0.814 / Accuracy: 0.965
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.928 / Recall: 0.821 / Accuracy: 0.966
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.032 / Accuracy: 0.864
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.933 / Recall: 0.808 / Accuracy: 0.965
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.901 / Recall: 0.821 / Accuracy: 0.962
Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.006 / Accuracy: 0.861
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.915 / Recall: 0.827 / Accuracy: 0.965
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.908 / Recall: 0.821 / Accur

KeyboardInterrupt: 

### Exploring parameter settings using GridSearchCV

In [ ]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]